# Configuring InstructLab

<ul>
<li>Contributors: InstructLab team and IBM Research Technology Education team
<li>Contact for questions and technical support: IBM.Research.JupyterLab@ibm.com
<li>Provenance: IBM Research
<li>Version: 1.0.9
<li>Release date: 2024-11-14
<li>Compute requirements: GPU: estimated 4 to 15 minutes (up to 4 min for cell 6, 8 min for cell 7. and 5 min for cell 8)
<li>Memory requirements: 16 GB
<li>Notebook set: InstructLab
</ul>

In [14]:
!ls

config.yaml  models  sample_data  taxonomy


# Summary
This notebook set demonstrates InstructLab, an open source AI project that facilitates knowledge and skills contributions to Large Language Models (LLMs). InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081). The open source InstructLab repository is available [here](https://github.com/instructlab/instructlab) and provides additional documentation on using InstructLab.

InstructLab  can be instantiated in several different forms, depending on the processing capabilities available. InstructLab can take the form of an open source installation or a Red Hat AI InstructLab installation. The open source installation can be run on a range of hardware from a laptop to a build your own (BYO) server instance running on a Virtual Machine (VM). The below figure shows the different available instantiations of InstructLab.

<img src="https://github.com/KenOcheltree/ilab-colab/blob/main/data/images/experiences.png?raw=1" width="800">

In this notebook set, we will be demonstrating both the open source version running on a VM server and Red Hat Enterprise Linux AI InstructLab running on an IBM Cloud Server.

The open source version running on a server is demonstrated in the following notebooks that are run sequentially:
- [Configuring InstructLab](./00_configuring_InstructLab.ipynb)
- [Training with InstructLab](./01_training_with_InstructLab.ipynb)
- [Inferencing with InstructLab](./02_inferencing_with_InstructLab.ipynb)

The Red Hat Enterprise Linux AI InstructLab is demonstrated running as a service in the IBM Cloud by running the following notebooks:
- [Configuring InstructLab](./00_configuring_InstructLab.ipynb)
- [Training with Red Hat AI InstructLab Service](./03_training_with_RH_AI_InstructLab_Service.ipynb)
- [Inferencing with Redhat-AI-InstructLab Trained Model](./04_inferencing_with_RH_AI_InstructLab_Service.ipynb)
**Note:** The **Configuring InstructLab** notebook is run before the other Red Hat AI InstructLab notebooks to ensure that the *granite 7b model* is installed for inference comparisons as the base model.

## Configuring InstructLab

This notebook demonstrates the configuration of InstructLab. The InstructLab method consists of three major components:
* **Taxonomy-driven data curation:**  The taxonomy is a set of training data curated by humans as examples of new knowledge and skills for the model.
* **Large-scale synthetic data generation:** A teacher model is used to generate new examples based on the seed training data. Recognizing that synthetic data can vary in quality, the InstructLab method adds an automated step to refine the example answers, ensuring they are grounded and safe.
* **Iterative model alignment tuning:** The model is retrained based on the synthetic data. The InstructLab method includes two tuning phases: knowledge tuning, followed by skill tuning.

In this notebook, we will demonstrate the following:
1. Checking the InstructLab installation
2, Configuring InstructLab for use
3. Installing InstructLab LLM models

**Note:** This notebook must be run within a GPU session. If you are not running with a GPU, please select **File->Hub Control Panel->Stop My Server**, then **Start My Server** and then select a GPU Session.

# Table of Contents
* <a href="#I0_preconfig">Step 0. Environment Preconfiguration</a>
* <a href="#I0_init">Step 1. Check the Starting Configuration</a>
* <a href="#I0_config">Step 2. Configure InstructLab</a>
* <a href="#I0_down">Step 3. Download Models</a>
* <a href="#I0_conclusion">Conclusion</a>
* <a href="#I0_learn">Learn more</a>

<a id="#I0_preconfig"></a>
# Step 0. Environment Preconfiguration
This step has already been performed for this JupyterLab environment and requires no work by the user. This information is provided in case the user is setting up their own environment on their own laptop or server.

The full steps for a direct installation are [here](https://github.com/instructlab/instructlab).



In [1]:
!python --version

Python 3.11.11


In [2]:
command = f"""
pip uninstall instructlab<<EOF
Y
"""

# Using the ! operator to run the command
#!echo "Running ilab config init"
#!{command}

In [3]:
!pip install instructlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
 

<a id="#I0_check"></a>
# Step 1. Check the Starting Configuration

## 1.1 Check for a GPU

This code cell checks for a GPU in the configuration as is required to run the kernel.

If you receive an error about a GPU not in the configuration, preform the following:
1. Select File->Hub Control Panel.
1. On the Hub Control Panel, select the blue "Stop My Server" button.
1. Then select "Start My Server" and choose Session with a GPU.
1. Check at the upper right that the kernel being run is "conda-instructlab-latest". If not, select the kernel and switch to the correct kernel.

In [1]:
# standard imports
import os
import torch

# torch and cuda version check
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

if torch.cuda.is_available() is False:
    print("No GPU in configuration")
else:
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    print("GPU is Available")

torch:  2.5 ; cuda:  cu124
No GPU in configuration


## 1.2 Check InstructLab Version and GPU Offload

Check that InstructLab version 0.23.1 is installed properly and is configured for using a GPU.

The first line from 'InstructLab' section should read
```
instructlab.version: 0.23.1
```

In [2]:
!ilab system info

Platform:
  sys.version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
  sys.platform: linux
  os.name: posix
  platform.release: 6.1.85+
  platform.machine: x86_64
  platform.node: f373b93e1909
  platform.python_version: 3.11.11
  os-release.ID: ubuntu
  os-release.VERSION_ID: 22.04
  os-release.PRETTY_NAME: Ubuntu 22.04.4 LTS
  memory.total: 12.67 GB
  memory.available: 11.24 GB
  memory.used: 1.12 GB

InstructLab:
  instructlab.version: 0.23.1
  instructlab-dolomite.version: 0.2.0
  instructlab-eval.version: 0.5.1
  instructlab-quantize.version: 0.1.0
  instructlab-schema.version: 0.4.2
  instructlab-sdg.version: 0.7.0
  instructlab-training.version: 0.7.0

Torch:
  torch.version: 2.5.1+cu124
  torch.backends.cpu.capability: AVX2
  torch.version.cuda: 12.4
  torch.version.hip: None
  torch.cuda.available: False
  torch.backends.cuda.is_built: True
  torch.backends.mps.is_built: False
  torch.backends.mps.is_available: False

llama_cpp_python:
  llama_cpp_python.version: 0.3.2
 

<a id="#I0_config"></a>
# Step 2. Configure InstructLab

## 2.1 Create InstructLab config file
The InstructLab configuration is captured in the *config.yaml* file. This step creates the config.yaml file and sets:
- **taxomony_path = taxonomy** - the root location of the taxonomy is set to the taxonomy folder in instructlab-latest
- **model_path = models/merlinite-7b-lab-Q4_K_M.gguf** - the default model is set to merlinite

**Note:** The default directories for InstructLab are the following. If you initialize InstructLab on your own system, it will default to the following:
* **Downloaded Models:**  ~/.cache/instructlab/models/ - Contains all downloaded large language models, including the saved output of ones you generate with ilab.
* **Synthetic Data:** ~/.local/share/instructlab/datasets/ - Contains data output from the SDG phase, built on modifications to the taxonomy repository.
* **Taxonomy:** ~/.local/share/instructlab/taxonomy/ - Contains the skill and knowledge data.
* **Training Output:** ~/.local/share/instructlab/checkpoints/ - Contains the output of the training process.
* **config.yaml:** ~/.config/instructlab/config.yaml - Contains the config.yaml file

In [3]:
import shutil
base_dir="/root/"
model_dir="models"
#Choose the base model as granite or mixtral
# Choose as quantized granite model
model_name="granite-7b-lab-Q4_K_M.gguf"

model_path = os.path.join(model_dir, model_name)
taxonomy_path='taxonomy'

# Define the file name
file_name = "config.yaml"
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"ilab was already initialized. {file_name} has been deleted. Reinitialized")
else:
    print(f"ilab was not initialized yet. {file_name} does not exist.")

#Remove old data
if os.path.exists("taxonomy"):
    print("removing taxonomy")
    shutil.rmtree("taxonomy")
if os.path.exists(base_dir+".cache/instructlab"):
    print("removing " + base_dir+".cache/instructlab")
    shutil.rmtree(base_dir+".cache/instructlab")
if os.path.exists(base_dir+".config/instructlab"):
    print("removing " + base_dir+".config/instructlab")
    shutil.rmtree(base_dir+".config/instructlab")
if os.path.exists(base_dir+".local/share/instructlab"):
    print("removing " + base_dir+".local/share/instructlab")
    shutil.rmtree(base_dir+".local/share/instructlab")

print(f"ilab model is {model_path}.")
print('#############################################################')
print(' ')

command = f"""
ilab config init<<EOF
{taxonomy_path}
Y
{model_path}
4
1
EOF
"""

# Using the ! operator to run the command
!echo "Running ilab config init"
!{command}

ilab was not initialized yet. config.yaml does not exist.
removing taxonomy
removing /root/.cache/instructlab
removing /root/.config/instructlab
removing /root/.local/share/instructlab
ilab model is models/granite-7b-lab-Q4_K_M.gguf.
#############################################################
 
Running ilab config init

----------------------------------------------------
         Welcome to the InstructLab CLI
  This guide will help you to setup your environment
----------------------------------------------------

Please provide the following values to initiate the environment [press 'Enter' for default options when prompted]
Path to taxonomy repo [/root/.local/share/instructlab/taxonomy]: `taxonomy` seems to not exist or is empty.
Should I clone https://github.com/instructlab/taxonomy.git for you? [Y/n]: Cloning https://github.com/instructlab/taxonomy.git...
Path to your model [/root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf]: 
Generating config file:
    /root/.config/

## 2.2 Display the config.yaml file
We examine the base configuration for identifying parameters for changing in the next step.

In [5]:
#to copy config.yaml to local directory
!cp {base_dir}.config/instructlab/config.yaml .
!cat config.yaml

# Chat configuration section.
chat:
  # Predefined setting or environment that influences the behavior and responses of
  # the chat assistant. Each context is associated with a specific prompt that
  # guides the assistant on how to respond to user inputs. Available contexts:
  # default, cli_helper.
  # Default: default
  context: default
  # Directory where chat logs are stored.
  # Default: /root/.local/share/instructlab/chatlogs
  logs_dir: /root/.local/share/instructlab/chatlogs
  # The maximum number of tokens that can be generated in the chat completion. Be
  # aware that larger values use more memory.
  # Default: None
  max_tokens:
  # Model to be used for chatting with.
  # Default: /root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf
  model: models/granite-7b-lab-Q4_K_M.gguf
  # Filepath of a dialog session file.
  # Default: None
  session:
  # Controls the randomness of the model's responses. Lower values make the output
  # more deterministic, while higher values 

## 2.3 Customize LLM Models and copy to notebook for use

This cell changes the models to use for the generate stage. The mistral model as the teacher model in the generate step and as the student model to be trained.

If you want to customize other models for generation or the training phase, you would specify the models in this step.

This step specifies that the models to be used will be from this notebook's models directory.

In [6]:
#Use ruamel.yaml to load the yaml file to preserve comments
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open('config.yaml', 'r') as file:
    config = yaml.load(file)

#Upate to use the same models and just change the directory
teacher_model_path = "models/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
base_model_path = "models/instructlab/granite-7b-lab"
#judge_model_path = "models/prometheus-eval/prometheus-8x7b-v2.0"

#config['evaluate']['mt_bench']['judge_model'] = judge_model_path
#config['evaluate']['mt_bench_branch']['judge_model'] = judge_model_path
config['generate']['model'] = teacher_model_path
config['generate']['teacher']['model_path']= teacher_model_path
#config['train']['phased_mt_bench_judge']=judge_model_path

# Save the updated config.yaml file
yaml.default_flow_style=False
with open('config.yaml', 'w') as file:
    yaml.dump(config, file)

#copy the config file to the .config/instructlab/ where it is used by InstructLab
!cp config.yaml {base_dir}.config/instructlab/

print("Updated config.yaml successfully.\n")
!cat config.yaml

Updated config.yaml successfully.

# Chat configuration section.
chat:
  # Predefined setting or environment that influences the behavior and responses of
  # the chat assistant. Each context is associated with a specific prompt that
  # guides the assistant on how to respond to user inputs. Available contexts:
  # default, cli_helper.
  # Default: default
  context: default
  # Directory where chat logs are stored.
  # Default: /root/.local/share/instructlab/chatlogs
  logs_dir: /root/.local/share/instructlab/chatlogs
  # The maximum number of tokens that can be generated in the chat completion. Be
  # aware that larger values use more memory.
  # Default: None
  max_tokens:
  # Model to be used for chatting with.
  # Default: /root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf
  model: models/granite-7b-lab-Q4_K_M.gguf
  # Filepath of a dialog session file.
  # Default: None
  session:
  # Controls the randomness of the model's responses. Lower values make the output
  # more 

<a id="#I0_down"></a>
# Step 3. Download Models
The models that will be used in the InstructLab processing are downloaded in this step. Additional steps can be added if other models are used in processing.

## 3.1. Download the merlinite and mistral-7b-instruct-v0.2.Q4_K_M models

The merlinite model will be used as the teacher model for the simple pipeline in the [Training with InstructLab](./01_training_with_InstructLab.ipynb) notebook.

The mistral-7b-instruct-v0.2.Q4_K_M model will be used as the teacher model for the full pipeline in the same notebook.

The granite07b-lab.gguf model is a quantized version oft eh granite-7b-lab model.

In [7]:
models_dir="models"
hf_token = ""
!ilab model download --hf-token {hf_token} --model-dir {models_dir}

INFO 2025-02-06 21:38:26,875 instructlab.model.download:77: Downloading model from Hugging Face:
    Model: instructlab/granite-7b-lab-GGUF@main
    Destination: models
INFO 2025-02-06 21:38:27,205 huggingface_hub.file_download:1536: Downloading 'granite-7b-lab-Q4_K_M.gguf' to 'models/.cache/huggingface/download/cBHY6fAjnFSRbyeHvtiPqRVG_SE=.6adeaad8c048b35ea54562c55e454cc32c63118a32c7b8152cf706b290611487.incomplete'
granite-7b-lab-Q4_K_M.gguf: 100% 4.08G/4.08G [00:46<00:00, 88.4MB/s]
Download complete. Moving file to models/granite-7b-lab-Q4_K_M.gguf
INFO 2025-02-06 21:39:13,528 huggingface_hub.file_download:1552: Download complete. Moving file to models/granite-7b-lab-Q4_K_M.gguf
INFO 2025-02-06 21:39:13,529 instructlab.model.download:289: 
ᕦ(òᴗóˇ)ᕤ instructlab/granite-7b-lab-GGUF model download completed successfully! ᕦ(òᴗóˇ)ᕤ

INFO 2025-02-06 21:39:13,529 instructlab.model.download:77: Downloading model from Hugging Face:
    Model: instructlab/merlinite-7b-lab-GGUF@main
    Destina

## 3.2. Optionally Download the granite 7b safe tensors model

Download the *granite-7b-lab* model. The  *granite-7b-lab* model is used as:
* The default base model for training in the [Training with InstructLab](./01_training_with_InstructLab.ipynb) notebook.
* The default base model for inferencing comparisons in the [Inferencing with InstructLab](./02_inferencing_with_InstructLab.ipynb) notebook.
* The base model for inferencing comparisons in the [Inferencing with Redhat-AI-InstructLab Trained Model](./04_inferencing_with_RH_AI_InstructLab_Service.ipynb) notebook.

In [ ]:
#!ilab model download --repository instructlab/granite-7b-lab --hf-token {hf_token} --model-dir {models_dir}

## 3.3. Optionally download the prometheus-8x7b-v2.0 model
The *prometheus-8x7b-v2.0* model is used as a judege model for multi-phase training and benchmark evaluation. This model is not required for simple or full training.

In [ ]:
#!ilab model download --repository prometheus-eval/prometheus-8x7b-v2.0 --hf-token {hf_token} --model-dir {models_dir}

<a id="I0_conclusion"></a>
# Conclusion

This notebook demonstrated setting up the InstructLab environment to be ready for introducing datasets for data generation, training and model creation.

<a id="I0_learn"></a>
# Learn More

Proceed to run the [Training with InstructLab](./01_training_with_InstructLab.ipynb) notebook to introduce datasets, perform synthetic data generation and train InstructLab models.

InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081).

This notebook is based on the open source InstructLab CLI repository available [here](https://github.com/instructlab/instructlab).

Contact us by email to ask questions, discuss potential use cases, or schedule a technical deep dive. The contact email is IBM.Research.JupyterLab@ibm.com.

© 2025 IBM Corporation